In [2]:

from dplearning_part.dplearning_second_part.limu_dplearning.utils.useful_func import *

In [60]:
batch_size, num_steps = 32, 35
##train_iter 返回的是每个字母的索引
train_iter, vocab = load_data_time_machine(batch_size, num_steps)
# train_iter中 每一个batch都是32*35 且有两个一个是x一个是y

In [76]:
def get_parameter(input_size, hidden_size, device='cpu'):
    output_size = input_size

    def normal(shape, std=0.01):
        return torch.randn(shape, device=device) * std

    W_xh = normal(shape=[input_size, hidden_size])
    W_hh = normal(shape=[hidden_size, hidden_size])
    B_h = torch.zeros(hidden_size, device=device)
    W_hq = normal(shape=[hidden_size, output_size])
    B_q = torch.zeros(output_size, device=device)
    parameters = [W_xh, W_hh, B_h, W_hq, B_q]
    for parameter in parameters:
        parameter.required_grad=True
    return W_xh, W_hh, B_h, W_hq, B_q

# 输入的x应该是32*35*embed
# 每一个x应该是 batch_size * embed_size
# hidden_state 应该是 batch_size*hidden_size
def rnn(X,state,parameters):

    W_xh, W_hh, B_h, W_hq, B_q=parameters
    hidden_state,=state
    
    outputs=[]
    for x in X:
        hidden_state=F.tanh(x@W_xh+hidden_state@W_hh+B_h)
        output=hidden_state@W_hq+B_q
    outputs.append(output)
    return torch.stack(outputs,dim=0),(hidden_state,)
        
    
class RNN(nn.Module):
    def __init__(self, input_size,num_steps, batch_size ,hidden_size):
        super(RNN, self).__init__()
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.parameters = get_parameter(input_size, hidden_size)
        self.init_hidden_state=(torch.zeros(self.batch_size,hidden_size),)
    def forward(self, X):
        X = F.one_hot(X.T, self.input_size).type(torch.float32)
        return rnn(X,self.init_hidden_state,self.parameters)
        
    

In [80]:
net=RNN(input_size=28,num_steps=35,batch_size=32,hidden_size=16)

In [81]:
net(next(iter(train_iter))[0])

(tensor([[[ 4.7141e-04,  6.7534e-05, -3.5947e-04, -9.0847e-05, -5.2319e-04,
           -4.3655e-04,  2.5396e-04, -1.5033e-04,  8.3949e-05,  8.3960e-05,
           -2.3564e-04,  5.9147e-04,  7.8115e-04,  1.6432e-04, -6.7499e-05,
            6.0294e-05, -3.1635e-04, -2.5506e-04,  8.7381e-05,  3.5010e-04,
           -1.2410e-04,  2.3190e-05,  4.6558e-04, -8.5032e-04,  7.7682e-04,
            3.4712e-04,  2.1042e-04,  5.6925e-04],
          [-6.2374e-05,  3.9052e-04,  3.3414e-04,  7.2677e-07, -7.7456e-05,
           -4.0928e-04, -1.8414e-04, -1.0339e-04,  1.0353e-04,  2.4257e-04,
            2.4365e-04, -3.3629e-04,  5.9787e-04,  5.2964e-04,  1.0329e-04,
           -6.6628e-05,  1.2461e-04,  1.5060e-04, -3.4280e-04, -2.9823e-04,
            7.2561e-05,  3.3473e-04, -2.0468e-04,  5.8046e-04, -4.4484e-04,
           -2.4252e-04, -1.6081e-04, -8.9993e-05],
          [ 4.0983e-04, -1.6165e-04, -4.4928e-04, -3.3807e-05,  4.2238e-04,
            5.1258e-05,  2.5473e-04,  6.2011e-04, -1.0663e-04,

In [83]:
nn.GRU

torch.nn.modules.rnn.GRU